In [3]:
import pandas as pd
import re

# === 1. CSV-Datei laden (korrekter Pfad & Komma als Trennzeichen) ===
file_path = "/Users/anniemcpherson/Library/CloudStorage/OneDrive-FHNW/FHNW FS-25/CDA/rel_gemeinde.csv"
df = pd.read_csv(file_path, sep=",", encoding="utf-8")

# === 2. Leere/unnötige Spalten entfernen ===
df = df.loc[:, ~df.columns.str.match(r"^Unnamed")]

# === 3. In Long-Format bringen ===
id_vars = ['Regions_ID', 'Regionsname', 'Kanton']
value_vars = [col for col in df.columns if col.startswith("y_")]

df_long = pd.melt(df,
                  id_vars=id_vars,
                  value_vars=value_vars,
                  var_name="Jahr",
                  value_name="Religion")

# === 4. Jahr bereinigen (z. B. y_1850 → 1850 als int) ===
df_long["Jahr"] = df_long["Jahr"].str.extract(r"y_(\d+)").astype(int)

# === 5. Encoding-Fehler beheben (‚â• → ≥) ===
df_long["Religion"] = df_long["Religion"].str.replace("‚â•", "≥", regex=False)

# === 6. Nur gültige Kantone (z. B. ZH, BE, AG etc.) ===
df_long["Kanton"] = df_long["Kanton"].astype(str).str.strip()
df_long = df_long[df_long["Kanton"].str.match(r"^[A-Z]{2}$")]

# === 7. Aggregation: häufigste Religion pro Kanton & Jahr ===
df_kanton_agg = (
    df_long
    .groupby(["Kanton", "Jahr"])["Religion"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    .reset_index()
)

# === 8. Funktion zur Gruppenzuweisung (verfeinert) ===
def religion_group(text):
    if pd.isna(text):
        return "Other"
    
    text = text.lower()
    is_kat = "katholisch" in text
    is_ref = "reformiert" in text

    # Prozent extrahieren (z. B. "70–80%" oder "≥ 65%")
    match = re.search(r"(\d{1,2},?\d*) ?[-–] ?(\d{1,2},?\d*)", text)
    if match:
        lower = float(match.group(1).replace(",", "."))
        upper = float(match.group(2).replace(",", "."))
        avg = (lower + upper) / 2
    else:
        match_single = re.search(r"≥ ?(\d{1,2},?\d*)", text)
        if match_single:
            avg = float(match_single.group(1).replace(",", "."))
        else:
            return "Other"

    # Einteilung nach Schwellenwerten (65 % für "Stark", 50 % für "Eher")
    if is_kat:
        if avg >= 65:
            return "Stark_Kat"
        elif avg >= 50:
            return "Eher_Kat"
    elif is_ref:
        if avg >= 65:
            return "Stark_Ref"
        elif avg >= 50:
            return "Eher_Ref"

    return "Other"

# === 9. Pivotieren: Jahre als Spalten ===
df_pivot = df_kanton_agg.pivot(index="Kanton", columns="Jahr", values="Religion")

# === 10. Neue Gruppenzuweisung je Zelle ===
df_grouped = pd.DataFrame(index=df_pivot.index)
for jahr in df_pivot.columns:
    df_grouped[f"{jahr}_Gruppe"] = df_pivot[jahr].apply(religion_group)

# === 11. Vorschau oder Speichern ===
# Vorschau (z. B. in Jupyter automatisch sichtbar)
display(df_grouped)

# Optional speichern
# df_grouped.to_csv("Kantonale_Religionsgruppen_differenziert.csv")


,1850_Gruppe,1860_Gruppe,1870_Gruppe,1880_Gruppe,1890_Gruppe,1900_Gruppe,1910_Gruppe,1920_Gruppe,1930_Gruppe,1940_Gruppe,1950_Gruppe,1960_Gruppe,1970_Gruppe,1980_Gruppe,1990_Gruppe,2000_Gruppe,2014_Gruppe
Kanton,,,,,,,,,,,,,,,,,
AG,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Kat,Stark_Ref,Stark_Ref,Stark_Kat,Other,Other
AI,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat
AR,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Other,Other,Other
BE,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref
BL,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref
BS,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Stark_Ref,Other,Stark_Ref,Other,Other,Other,Other
FR,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat
GE,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Kat,Stark_Ref,Other,Stark_Kat,Other,Other,Other,Other,Other,Other
GL,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other
